In [1]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules
import pickle

In [2]:
df = pd.read_pickle("../data/processed.pkl")
categorical_columns = df.select_dtypes(include=['category']).columns

In [3]:
with open("../data/data_element_details.pkl", "rb") as file:
    data_element_details = pickle.load(file)

In [4]:
keep_cols = []
for value in data_element_details.values():
    column_name = f"{value['Label']} ({value['SAS Variable Name']})"
    if value['Section Name'] in ["Health Care Access", "Chronic Health Conditions"] and column_name in df.columns:
        keep_cols.append(column_name)
df = df[keep_cols]

KeyError: "None of [Index(['What is Primary Source of Health Insurance?',\n       'Have Personal Health Care Provider?', 'Could Not Afford To See Doctor',\n       'Length of time since last routine checkup',\n       'Ever Diagnosed with Heart Attack',\n       'Ever Diagnosed with Angina or Coronary Heart Disease',\n       'Ever Diagnosed with a Stroke', 'Ever Told Had Asthma',\n       'Still Have Asthma',\n       '(Ever told) (you had) skin cancer that is not melanoma?',\n       '(Ever told) (you had)  melanoma or any other types of cancer?',\n       'Ever told you had C.O.P.D. emphysema or chronic bronchitis?',\n       '(Ever told) you had a depressive disorder',\n       'Ever told you have kidney disease?', 'Told Had Arthritis',\n       '(Ever told) you had diabetes'],\n      dtype='object')] are in the [columns]"

In [ ]:
df

In [ ]:
# df = df[categorical_columns]
transactions = []
for _, row in df.iterrows():
    transaction = [f"{col}:{val}" for col, val in zip(df.columns, row) if pd.notna(val) and not col.startswith("BLANK")]
    transactions.append(transaction)

In [ ]:
# Encode the transactions
te = TransactionEncoder()
te_ary = te.fit_transform(transactions)
encoded_df = pd.DataFrame(te_ary, columns=te.columns_)

In [ ]:
# Apply FP-Growth algorithm
frequent_itemsets = fpgrowth(encoded_df, min_support=0.1, use_colnames=True, max_len=5)

In [ ]:
# Generate association rules
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)

In [ ]:
# Print the frequent itemsets and association rules
print("Frequent Itemsets:")
print(frequent_itemsets)
print("\nAssociation Rules:")
rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

In [ ]:
rules['antecedents'][0]

In [ ]:
rules

In [ ]:
rules[rules['consequents'] == frozenset("What is Primary Source of Health Insurance?:1 - A plan purchased through an employer or union (including plans purchased through another person´s employer)")]

In [ ]:
with open('../data/frequent_itemsets.pickle', 'wb') as handle:
    pickle.dump(frequent_itemsets, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../data/association_rules.pickle', 'wb') as handle:
    pickle.dump(rules, handle, protocol=pickle.HIGHEST_PROTOCOL)